In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string

In [13]:


#read dataset
spam_df = pd.read_csv('spam.csv', encoding="ISO-8859-1")

#subset and rename columns
spam_df = spam_df[['v1', 'v2']]
spam_df.rename(columns={'v1': 'spam', 'v2': 'text'}, inplace=True)

#convert spam column to binary
spam_df.spam = spam_df.spam.apply(lambda s: True if s=='spam' else False)
spam_df.text = spam_df.text.apply(lambda t: t.lower().translate(str.maketrans('', '', string.punctuation)))

#shuffle
spam_df = spam_df.sample(frac=1)



In [17]:
#spam
for t in spam_df[spam_df.spam == True].iloc[:5].text:
    print(t)
    print('-------')


you have 1 new voicemail please call 08719181503
-------
you are a winner you have been specially selected to receive å£1000 cash or a å£2000 award speak to a live operator to claim call 087123002209am7pm cost 10p
-------
pls stop bootydelious 32f is inviting you to be her friend reply yes434 or no434 see her wwwsmsacubootydelious stop send stop frnd to 62468
-------
private your 2003 account statement for 07808247860 shows 800 unredeemed s i m points call 08719899229 identifier code 40411 expires 061104
-------
got what it takes 2 take part in the wrc rally in oz u can with lucozade energy text rally le to 61200 25p see packs or lucozadecoukwrc  itcould be u
-------


In [18]:
#not spam
for t in spam_df[spam_df.spam == False].iloc[:5].text:
    print(t)
    print('-------')



sir i am waiting for your mail
-------
now only i reached home   i am very tired now  i will come tomorro
-------
hello thanx for taking that call i got a job starts on monday
-------
so anyways you can just go to your gym or whatever my love smiles i hope your ok and having a good day babe  i miss you so much already
-------
i can take you at like noon
-------


In [19]:
#get training set
train_spam_df = spam_df.iloc[:int(len(spam_df)*0.7)]

#get testing set
test_spam_df = spam_df.iloc[int(len(spam_df)*0.7):]

In [21]:
FRAC_SPAM_TEXTS = train_spam_df.spam.mean()
print(FRAC_SPAM_TEXTS)
#percentage of spam text

0.13282051282051283


In [25]:
# all data
# converting into  long sentence containg spam/nonspam words spereated by ' ' and split
train_spam_words = ' '.join(train_spam_df[train_spam_df.spam == True].text).split(' ')
train_non_spam_words = ' '.join(train_spam_df[train_spam_df.spam == False].text).split(' ')

common_words = set(train_spam_words).intersection(set(train_non_spam_words))

train_spam_bow = dict() 
#storing percentage of occurence of spam for each word
train_spam_bow = dict() 
for w in common_words:
    if w not in train_spam_bow:
        train_spam_bow[w] = train_spam_words.count(w) / len(train_spam_words)

train_non_spam_bow = dict()
for w in common_words:
    if w not in train_non_spam_bow:
        train_non_spam_bow[w] = train_non_spam_words.count(w) / len(train_non_spam_words)



In [27]:
train_spam_bow

{'': 0.013593951093951093,
 'suzy': 8.043758043758043e-05,
 'entered': 0.00032175032175032174,
 'immediately': 0.00016087516087516087,
 'down': 8.043758043758043e-05,
 'f': 0.00024131274131274132,
 'may': 0.00040218790218790216,
 'notice': 0.00016087516087516087,
 'use': 0.00032175032175032174,
 'female': 8.043758043758043e-05,
 '2day': 0.00024131274131274132,
 'slow': 8.043758043758043e-05,
 'uve': 0.00032175032175032174,
 'goto': 0.00024131274131274132,
 'london': 0.00024131274131274132,
 'day': 0.0008848133848133849,
 'chance': 0.0016087516087516086,
 'password': 8.043758043758043e-05,
 'details': 0.00048262548262548264,
 'theres': 0.00024131274131274132,
 'correct': 0.00024131274131274132,
 'who': 0.002171814671814672,
 'blow': 0.00016087516087516087,
 'old': 0.00016087516087516087,
 'special': 0.0006435006435006435,
 'summer': 0.0006435006435006435,
 'sent': 0.00040218790218790216,
 'tickets': 0.00016087516087516087,
 'alone': 0.00024131274131274132,
 'video': 0.001608751608751608

In [29]:
train_non_spam_bow

{'': 0.02261865524060646,
 'suzy': 2.059986816084377e-05,
 'entered': 8.239947264337508e-05,
 'immediately': 4.119973632168754e-05,
 'down': 0.000720995385629532,
 'f': 4.119973632168754e-05,
 'may': 0.0004943968358602505,
 'notice': 6.179960448253131e-05,
 'use': 0.0004943968358602505,
 'female': 4.119973632168754e-05,
 '2day': 6.179960448253131e-05,
 'slow': 0.0001441990771259064,
 'uve': 2.059986816084377e-05,
 'goto': 4.119973632168754e-05,
 'london': 4.119973632168754e-05,
 'day': 0.002574983520105471,
 'chance': 0.0002059986816084377,
 'password': 4.119973632168754e-05,
 'details': 0.0003089980224126566,
 'theres': 0.00037079762689518785,
 'correct': 0.00012359920896506263,
 'who': 0.0009063941990771259,
 'blow': 2.059986816084377e-05,
 'old': 0.00026779828609096903,
 'special': 0.0005149967040210942,
 'summer': 2.059986816084377e-05,
 'sent': 0.0008651944627554384,
 'tickets': 0.00012359920896506263,
 'alone': 0.00018539881344759393,
 'video': 2.059986816084377e-05,
 'st': 0.000

In [34]:
def predict_text(t, verbose=False):
    #if some word doesnt appear in either spam or non-spam BOW, disregard it
    valid_words = [w for w in t if w in train_spam_bow]
    
    #get the probabilities of each valid word showing up in spam and non-spam BOW
    spam_probs = [train_spam_bow[w] for w in valid_words]
    non_spam_probs = [train_non_spam_bow[w] for w in valid_words]
    
    #print probs if requested
    if verbose:
        data_df = pd.DataFrame()
        data_df['word'] = valid_words
        data_df['spam_prob'] = spam_probs
        data_df['non_spam_prob'] = non_spam_probs
        data_df['ratio'] = [s/n if n > 0 else np.inf for s,n in zip(spam_probs, non_spam_probs)]
        print(data_df)
     
    #calculate spam score as sum of logs for all probabilities
    spam_score = sum([np.log(p) for p in spam_probs]) + np.log(FRAC_SPAM_TEXTS)
    
    #calculate non-spam score as sum of logs for all probabilities
    non_spam_score = sum([np.log(p) for p in non_spam_probs]) + np.log(1-FRAC_SPAM_TEXTS)
    
    #if verbose, report the two scores
    if verbose:
        print('Spam Score: %s'%spam_score)
        print('Non-Spam Score: %s'%non_spam_score)
   
    #if spam score is higher, mark this as spam
    return (spam_score >= non_spam_score)




In [32]:


def predict_text(t, verbose=False):
    #if some word doesnt appear in either spam or non-spam BOW, disregard it
    valid_words = [w for w in t if w in train_spam_bow]
    
    #get the probabilities of each valid word showing up in spam and non-spam BOW
    spam_probs = [train_spam_bow[w] for w in valid_words]
    non_spam_probs = [train_non_spam_bow[w] for w in valid_words]
    
    #print probs if requested
    if verbose:
        data_df = pd.DataFrame()
        data_df['word'] = valid_words
        data_df['spam_prob'] = spam_probs
        data_df['non_spam_prob'] = non_spam_probs
        data_df['ratio'] = [s/n if n > 0 else np.inf for s,n in zip(spam_probs, non_spam_probs)]
        print(data_df)
     
    #calculate spam score as sum of logs for all probabilities
    spam_score = sum([np.log(p) for p in spam_probs]) + np.log(FRAC_SPAM_TEXTS)
    
    #calculate non-spam score as sum of logs for all probabilities
    non_spam_score = sum([np.log(p) for p in non_spam_probs]) + np.log(1-FRAC_SPAM_TEXTS)
    
    #if verbose, report the two scores
    if verbose:
        print('Spam Score: %s'%spam_score)
        print('Non-Spam Score: %s'%non_spam_score)
   
    #if spam score is higher, mark this as spam
    return (spam_score >= non_spam_score)




In [41]:
text="Hi Gogopower,With your full team on Notion, you'll be able to break down silos and work faster, together. Is your full team on Notion yet?Invite your teammates "
predict_text(text.split(), verbose=True)

  word  spam_prob  non_spam_prob     ratio
0   it    0.00177       0.008467  0.209014
Spam Score: -8.355743209107368
Non-Spam Score: -4.914141970311236


False